In [ ]:
!pip install kagglehub

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("banuprasadb/visdrone-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/visdrone-dataset


In [ ]:
import os

# Step 2: Define source and target paths
src_base = os.path.join(path, "VisDrone_Dataset")
target_base = "/content/rf-detr/data/visdrone"

In [ ]:
import shutil
import json
from PIL import Image

def copy_and_resize_images(src_dir, dst_dir, resize_to=(640, 640)):
    os.makedirs(dst_dir, exist_ok=True)
    for file in os.listdir(src_dir):
        if file.endswith('.jpg'):
            src_path = os.path.join(src_dir, file)
            dst_path = os.path.join(dst_dir, file)
            try:
                with Image.open(src_path) as img:
                    resized_img = img.resize(resize_to)
                    resized_img.save(dst_path)
            except Exception as e:
                print(f"Error processing image {file}: {e}")

def convert_yolo_to_coco(images_dir, labels_dir, output_json_path):
    coco_format = {
        "images": [],
        "annotations": [],
        "categories": []
    }

    categories = [
        {"id": 0, "name": "pedestrian", "supercategory": "person"},
        {"id": 1, "name": "person", "supercategory": "person"},
        {"id": 2, "name": "bicycle", "supercategory": "vehicle"},
        {"id": 3, "name": "car", "supercategory": "vehicle"},
        {"id": 4, "name": "van", "supercategory": "vehicle"},
        {"id": 5, "name": "truck", "supercategory": "vehicle"},
        {"id": 6, "name": "tricycle", "supercategory": "vehicle"},
        {"id": 7, "name": "awning-tricycle", "supercategory": "vehicle"},
        {"id": 8, "name": "bus", "supercategory": "vehicle"},
        {"id": 9, "name": "motor", "supercategory": "vehicle"}
    ]
    coco_format["categories"] = categories

    image_id = 0
    annotation_id = 0

    for label_file in os.listdir(labels_dir):
        if label_file.endswith(".txt"):
            image_file = os.path.splitext(label_file)[0] + ".jpg"
            image_path = os.path.join(images_dir, image_file)

            if not os.path.exists(image_path):
                print(f"Warning: Image not found for label file {label_file}")
                continue

            try:
                with Image.open(image_path) as img:
                    width, height = img.size
            except Exception as e:
                print(f"Error opening image {image_file}: {e}")
                continue

            coco_format["images"].append({
                "id": image_id,
                "file_name": image_file,
                "width": width,
                "height": height
            })

            with open(os.path.join(labels_dir, label_file), "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        class_id, center_x, center_y, bbox_width, bbox_height = map(float, parts)

                        x_center = center_x * width
                        y_center = center_y * height
                        w = bbox_width * width
                        h = bbox_height * height

                        x_min = x_center - w / 2
                        y_min = y_center - h / 2

                        if int(class_id) >= 0 and int(class_id) < 10:
                            coco_format["annotations"].append({
                                "id": annotation_id,
                                "image_id": image_id,
                                "category_id": int(class_id),
                                "bbox": [x_min, y_min, w, h],
                                "area": w * h,
                                "iscrowd": 0
                            })
                            annotation_id += 1
                        else:
                            print(f"Ignoring class_id {int(class_id)} in {label_file}")

            image_id += 1

    os.makedirs(os.path.dirname(output_json_path), exist_ok=True)
    with open(output_json_path, "w") as f:
        json.dump(coco_format, f, indent=4)
    print(f"COCO annotations saved to {output_json_path}")

In [ ]:
base_input = "/kaggle/input/visdrone-dataset/VisDrone_Dataset"
train_img_in = os.path.join(base_input, "VisDrone2019-DET-train/images")
val_img_in = os.path.join(base_input, "VisDrone2019-DET-val/images")
test_img_in = os.path.join(base_input, "VisDrone2019-DET-test-dev/images")

train_lbl = os.path.join(base_input, "VisDrone2019-DET-train/labels")
val_lbl = os.path.join(base_input, "VisDrone2019-DET-val/labels")
test_lbl = os.path.join(base_input, "VisDrone2019-DET-test-dev/labels")

In [ ]:
base_output = "/kaggle/working"
train_img_out = os.path.join(base_output, "train/images")
val_img_out = os.path.join(base_output, "val/images")
test_img_out = os.path.join(base_output, "test-dev/images")

json_train = os.path.join(base_output, "annotations/instances_train.json")
json_val = os.path.join(base_output, "annotations/instances_val.json")
json_test = os.path.join(base_output, "annotations/instances_test.json")

In [ ]:
# Make sure the working directories exist
os.makedirs("/content/visdrone/train/images", exist_ok=True)
os.makedirs("/content/visdrone/val/images", exist_ok=True)
os.makedirs("/content/visdrone/test-dev/images", exist_ok=True)
os.makedirs("/content/visdrone/annotations", exist_ok=True)

# 1. Copy and resize all images to /content
copy_and_resize_images(train_img_in, "/content/visdrone/train/images")
copy_and_resize_images(val_img_in, "/content/visdrone/val/images")
copy_and_resize_images(test_img_in, "/content/visdrone/test-dev/images")

# 2. Convert YOLO to COCO using resized images and original labels
convert_yolo_to_coco(
    images_dir="/content/visdrone/train/images",
    labels_dir=train_lbl,
    output_json_path="/content/visdrone/annotations/instances_train.json"
)

convert_yolo_to_coco(
    images_dir="/content/visdrone/val/images",
    labels_dir=val_lbl,
    output_json_path="/content/visdrone/annotations/instances_val.json"
)

convert_yolo_to_coco(
    images_dir="/content/visdrone/test-dev/images",
    labels_dir=test_lbl,
    output_json_path="/content/visdrone/annotations/instances_test.json"
)

COCO annotations saved to /content/visdrone/annotations/instances_train.json
COCO annotations saved to /content/visdrone/annotations/instances_val.json
COCO annotations saved to /content/visdrone/annotations/instances_test.json


In [ ]:
# Create rf-detr data folder structure
os.makedirs(os.path.join(target_base, "train"), exist_ok=True)
os.makedirs(os.path.join(target_base, "val"), exist_ok=True)
os.makedirs(os.path.join(target_base, "annotations"), exist_ok=True)

# Copy resized images
shutil.copytree("/content/visdrone/train/images", os.path.join(target_base, "train"), dirs_exist_ok=True)
shutil.copytree("/content/visdrone/val/images", os.path.join(target_base, "val"), dirs_exist_ok=True)

# Copy COCO-formatted annotations
shutil.copyfile("/content/visdrone/annotations/instances_train.json", os.path.join(target_base, "annotations", "instances_train.json"))
shutil.copyfile("/content/visdrone/annotations/instances_val.json", os.path.join(target_base, "annotations", "instances_val.json"))


'/content/rf-detr/data/visdrone/annotations/instances_val.json'

In [ ]:
copy_and_resize_images(train_img_in, train_img_out)
copy_and_resize_images(val_img_in, val_img_out)
copy_and_resize_images(test_img_in, test_img_out)

convert_yolo_to_coco(train_img_out, train_lbl, json_train)
convert_yolo_to_coco(val_img_out, val_lbl, json_val)
convert_yolo_to_coco(test_img_out, test_lbl, json_test)

COCO annotations saved to /kaggle/working/annotations/instances_train.json
COCO annotations saved to /kaggle/working/annotations/instances_val.json
COCO annotations saved to /kaggle/working/annotations/instances_test.json


In [ ]:
# Step 4: Create necessary folders
os.makedirs(os.path.join(target_base, "train"), exist_ok=True)
os.makedirs(os.path.join(target_base, "val"), exist_ok=True)
os.makedirs(os.path.join(target_base, "annotations"), exist_ok=True)

In [ ]:
# Step 5: Copy images
import shutil

shutil.copytree(
    os.path.join(src_base, "VisDrone2019-DET-train/images"),
    os.path.join(target_base, "train"),
    dirs_exist_ok=True
)
shutil.copytree(
    os.path.join(src_base, "VisDrone2019-DET-val/images"),
    os.path.join(target_base, "val"),
    dirs_exist_ok=True
)

'/content/rf-detr/data/visdrone/val'

In [ ]:
# Step 6: Copy annotation files
# Use the path where the dataset was downloaded by kagglehub as the source for copying JSONs
json_src_base = "/root/.cache/kagglehub/datasets/banuprasadb/visdrone-dataset/versions/1/VisDrone_Dataset"

shutil.copyfile(
    "/content/visdrone/annotations/instances_train.json",
    os.path.join(target_base, "annotations", "instances_train.json")
)

shutil.copyfile(
    "/content/visdrone/annotations/instances_val.json",
    os.path.join(target_base, "annotations", "instances_val.json")
)

shutil.copyfile(
    "/content/visdrone/annotations/instances_test.json",
    os.path.join(target_base, "annotations", "instances_test.json")
)

# Assuming you also want to copy the test annotation file if it exists and is needed
# shutil.copyfile(
#     os.path.join(json_src_base, "instances_test.json"),
#     os.path.join(target_base, "annotations", "instances_test.json")
# )

'/content/rf-detr/data/visdrone/annotations/instances_test.json'

In [ ]:
import os
import shutil

# Define source and target paths (using variables defined earlier)
# src_base is defined in cell ejFI9nD5FcfF
# target_base is defined in cell ejFI9nD5FcfF

# Copy validation images to the 'valid' directory
# The source for validation images is the same as the 'val' images
shutil.copytree(
    os.path.join(src_base, "VisDrone2019-DET-val/images"),
    os.path.join(target_base, "valid"),
    dirs_exist_ok=True
)
print(f"Copied validation images to {os.path.join(target_base, 'valid')}")

# Copy test-dev images to the 'test' directory
shutil.copytree(
    os.path.join(src_base, "VisDrone2019-DET-test-dev/images"),
    os.path.join(target_base, "test"),
    dirs_exist_ok=True
)
print(f"Copied test-dev images to {os.path.join(target_base, 'test')}")

Copied validation images to /content/rf-detr/data/visdrone/valid
Copied test-dev images to /content/rf-detr/data/visdrone/test


In [ ]:
ls /content/rf-detr/data/visdrone/valid/ | wc -l

548


In [ ]:
!pip install rfdetr

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

from rfdetr import RFDETRBase
import shutil

# Create model instance
model = RFDETRBase()

# Define the paths for the generated COCO annotations
# Corrected source path to where the JSON files were generated
json_src_base = "/content/visdrone/annotations"

train_json_src = os.path.join(json_src_base, "instances_train.json")
val_json_src = os.path.join(json_src_base, "instances_val.json")
test_json_src = os.path.join(json_src_base, "instances_test.json") # Add test source path

# Define the target paths and names expected by the RF-DETR training script (based on the errors)
train_annotations_target_dir = "/content/rf-detr/data/visdrone/train"
val_annotations_target_dir = "/content/rf-detr/data/visdrone/val" # Keep this for now
valid_annotations_target_dir = "/content/rf-detr/data/visdrone/valid" # Add 'valid' directory target
test_annotations_target_dir = "/content/rf-detr/data/visdrone/test" # Add 'test' directory target

train_json_target = os.path.join(train_annotations_target_dir, "_annotations.coco.json")
val_json_target = os.path.join(val_annotations_target_dir, "_annotations.coco.json") # Keep this for now
valid_json_target = os.path.join(valid_annotations_target_dir, "_annotations.coco.json") # Add 'valid' directory target filename
test_json_target = os.path.join(test_annotations_target_dir, "_annotations.coco.json") # Add 'test' directory target filename


# Ensure the target directories exist
os.makedirs(train_annotations_target_dir, exist_ok=True)
os.makedirs(val_annotations_target_dir, exist_ok=True) # Ensure 'val' directory exists
os.makedirs(valid_annotations_target_dir, exist_ok=True) # Ensure 'valid' directory exists
os.makedirs(test_annotations_target_dir, exist_ok=True) # Ensure 'test' directory exists


# Copy and rename the annotation files to the expected locations
if os.path.exists(train_json_src):
    shutil.copyfile(train_json_src, train_json_target)
    print(f"Copied {train_json_src} to {train_json_target}")
else:
    print(f"Warning: {train_json_src} not found. Cannot copy train annotations.")

if os.path.exists(val_json_src):
    # Copy to both 'val' and 'valid' directories, assuming the script might look in either or both
    shutil.copyfile(val_json_src, val_json_target)
    print(f"Copied {val_json_src} to {val_json_target}")
    shutil.copyfile(val_json_src, valid_json_target)
    print(f"Copied {val_json_src} to {valid_json_target}")
else:
    print(f"Warning: {val_json_src} not found. Cannot copy val annotations.")

if os.path.exists(test_json_src): # Add copy for test annotations
    shutil.copyfile(test_json_src, test_json_target)
    print(f"Copied {test_json_src} to {test_json_target}")
else:
    print(f"Warning: {test_json_src} not found. Cannot copy test annotations.")

Loading pretrain weights
Copied /content/visdrone/annotations/instances_train.json to /content/rf-detr/data/visdrone/train/_annotations.coco.json
Copied /content/visdrone/annotations/instances_val.json to /content/rf-detr/data/visdrone/val/_annotations.coco.json
Copied /content/visdrone/annotations/instances_val.json to /content/rf-detr/data/visdrone/valid/_annotations.coco.json
Copied /content/visdrone/annotations/instances_test.json to /content/rf-detr/data/visdrone/test/_annotations.coco.json


In [ ]:
model = RFDETRBase(num_classes=10)


# Train the model using your converted VisDrone dataset
model.train(
    dataset_dir="/content/rf-detr/data/visdrone",  # path to COCO-formatted dataset
    epochs=10, # Keeping epochs low for testing
    batch_size=2,
    grad_accum_steps=4,
    lr=1e-4,
    output_dir="/content/rf-detr/output"  # where to save checkpoints and logs
)

Loading pretrain weights


reinitializing detection head with 90 classes


TensorBoard logging initialized. To monitor logs, use 'tensorboard --logdir /content/rf-detr/output' and open http://localhost:6006/ in browser.
Not using distributed mode
git:
  sha: N/A, status: clean, branch: N/A

Namespace(num_classes=10, grad_accum_steps=4, amp=True, lr=0.0001, lr_encoder=0.00015, batch_size=2, weight_decay=0.0001, epochs=10, lr_drop=100, clip_max_norm=0.1, lr_vit_layer_decay=0.8, lr_component_decay=0.7, do_benchmark=False, dropout=0, drop_path=0.0, drop_mode='standard', drop_schedule='constant', cutoff_epoch=0, pretrained_encoder=None, pretrain_weights='rf-detr-base.pth', pretrain_exclude_keys=None, pretrain_keys_modify_to_load=None, pretrained_distiller=None, encoder='dinov2_windowed_small', vit_encoder_num_layers=12, window_block_indexes=None, position_embedding='sine', out_feature_indexes=[2, 5, 8, 11], freeze_encoder=False, layer_norm=True, rms_norm=False, backbone_lora=False, force_no_pretrain=False, dec_layers=3, dim_feedforward=2048, hidden_dim=256, sa_nhe

OutOfMemoryError: CUDA out of memory. Tried to allocate 72.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 22.12 MiB is free. Process 73439 has 14.72 GiB memory in use. Of the allocated memory 14.38 GiB is allocated by PyTorch, and 204.12 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)